In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns


In [2]:
output_path = "../data/03_processed/merged_df.csv"
merged_df = pd.read_csv(output_path)
merged_df = merged_df.rename(columns={'total_year': 'HDD_total_year'})

# GDP per capita
merged_df['GDP_per_capita'] = merged_df['GDP_2022'] / merged_df['Population_2023']

# Delete Taiwan (temporal)
merged_df = merged_df[merged_df['Country'] != 'Taiwan']

np.random.seed(123)
merged_df['is_train'] = np.random.choice([1, 0], size=len(merged_df), p=[0.8, 0.2])



In [3]:
features = [
    "hdd_total_year", 
    "GDP_per_capita", 
    "Urbanization_Rate_2022",
    "latitude",
    "longitude",  
    "Paris_Agreement",
    "Subnational HDI", 
    "Educational index", 
    "Income index"
]

feature_abbreviations = {
    "HDD_total_year": "HDD",
    "GDP_per_capita": "GDP",
    "Urbanization_Rate_2022": "URB",
    "latitude": "Lat",
    "longitude": "Long",
    "Paris_Agreement": "Paris",
    "Subnational HDI": "HDI", 
    "Educational index": "EDU" , 
    "Income index": "Income"
}

target_columns = [
    "Residential EUI (kWh/m2/year)",
    "Non-residential EUI (kWh/m2/year)"
]

merged_df['EUI (kWh/m2/year)'] = merged_df[target_columns[0]] + merged_df[target_columns[1]]


In [4]:
# Function to calculate feature importance and return the importance table --> (THIS IS THE INPUT FOR THE PLOT)


def calculate_feature_importance(df, features, target_columns, feature_abbreviations):
    # Create the sum of the target columns
    total_eui = df[target_columns[0]] + df[target_columns[1]]
    
    # Prepare data
    X = df[features]
    y = total_eui
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Scale target (optional, but recommended for interpretation)
    y_scaled = StandardScaler().fit_transform(y.values.reshape(-1, 1)).ravel()
    
    # Train model
    model = LinearRegression()
    model.fit(X_scaled, y_scaled)
    
    # Get feature importance (absolute coefficients)
    importance = np.abs(model.coef_)
    
    # Create DataFrame with importance
    importance_df = pd.DataFrame({
        'Feature': [feature_abbreviations.get(f, f) for f in features],
        'Importance': importance
    })
    
    # Sort by importance
    importance_df = importance_df.sort_values('Importance', ascending=True)
    
    return importance_df


importance_df = calculate_feature_importance(merged_df, features, target_columns, feature_abbreviations)
importance_df.sort_values('Importance', ascending=False)


,Feature,Importance
0,hdd_total_year,0.515993
6,HDI,0.371646
3,Lat,0.202285
2,URB,0.116151
8,Income,0.114043
4,Long,0.088552
7,EDU,0.062440
5,Paris,0.024728
1,GDP,0.001931
